In [26]:
# import packages
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from joblib import dump

import time

# change working directory to where the data is
import os
os.chdir("/Users/moonbee/Cyto/Human_T_Cell_Profile/")

In [27]:
# read csv table with rows and columns
df = pd.read_csv("CD103_Lung_labeled.csv", index_col=0, header=0)
df.head()

,FSC-A,SSC-A,CD14,CD103,HLADR,CD20,CD8,CD4,CD3,CD45RA,CCR7,Population
0,262143.000,170898.53,1054.50,1688.580000,24736.14,30471.8180,1490.34000,6727.82000,1545.47990,55426.336,3071.97,CD20+ B cells
1,141144.950,68960.88,363.66,926.300000,15552.90,12555.6990,742.22000,4297.23970,704.89996,25495.120,1150.38,CD20+ B cells
2,104009.836,42376.08,12.54,923.939940,1407.78,133.5600,1564.67990,4456.23970,8291.31900,12832.359,11447.37,CD4+ T cells
3,100292.720,47255.28,126.54,-61.359997,5387.58,4061.9197,488.52000,708.07996,246.97998,9153.100,277.20,CD20+ B cells
4,107753.360,37875.36,198.36,377.599980,4177.80,3977.1199,292.63998,1387.53990,164.29999,9253.800,739.53,CD20+ B cells


In [28]:
# split the data into X and y
X = df.drop('Population', axis=1)
y = df['Population']

In [29]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# logistic regression

# start the timer
start_time_LR = time.time()

# create a pipeline with data standardization and logistic regression
pipeline_LR = Pipeline([
    ('scaler', MinMaxScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

# define the hyperparameters to be optimized
param_grid_LR = {
    'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logreg__penalty': ['l1', 'l2'],
    'logreg__solver': ['liblinear']
}

# create a GridSearchCV object
grid_search_LR = GridSearchCV(pipeline_LR, param_grid_LR, cv=5, scoring='accuracy', n_jobs=-1)

# fit the model
grid_search_LR.fit(X_train, y_train)

# keep best parameters
best_params = grid_search_LR.best_params_

# make prediction on the test set
y_pred = grid_search_LR.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# convert the report to a DataFrame
df_report = pd.DataFrame(report).transpose()

# add the column with the cell types
df_report.reset_index(inplace=True)
df_report.rename(columns={'index': 'Cell Type'}, inplace=True)

# stop the timer
end_time_LR = time.time()

# calculate the execution time
duration_LR = end_time_LR - start_time_LR

# print the execution time
print(f"Execution time: {int(duration_LR // 60)} minutes and {int(duration_LR % 60)} seconds" if duration_LR >= 60 else f"Execution time: {int(duration_LR)} seconds\n")

# print best hyperparameters
print("Best parameters: {}".format(best_params) + "\n")

# print classification report
print(df_report.to_string(index=False))

Execution time: 21 seconds

Best parameters: {'logreg__C': 100, 'logreg__penalty': 'l1', 'logreg__solver': 'liblinear'}

      Cell Type  precision   recall  f1-score      support
CD14+ Monocytes   0.976280 0.977862  0.977070  1852.000000
  CD20+ B cells   0.996255 0.996362  0.996308  9345.000000
   CD4+ T cells   0.996702 0.996264  0.996483  9101.000000
   CD8+ T cells   0.995120 0.987888  0.991490  2064.000000
    Other cells   0.963790 0.972594  0.968172  1642.000000
       accuracy   0.992543 0.992543  0.992543     0.992543
      macro avg   0.985629 0.986194  0.985905 24004.000000
   weighted avg   0.992565 0.992543  0.992551 24004.000000


In [31]:
# K-Nearest Neighbor

# start the timer
start_time_kNN = time.time()

# create a pipeline with data standardization and KNN
pipeline_kNN = Pipeline([
    ('scaler', MinMaxScaler()),
    ('knn', KNeighborsClassifier())
])

# define the hyperparameters to be optimized
param_grid_kNN = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

# create a GridSearchCV object
grid_search_kNN = GridSearchCV(pipeline_kNN, param_grid_kNN, cv=5, scoring='accuracy', n_jobs=-1)

# fit the model
grid_search_kNN.fit(X_train, y_train)

# keep best parameters
best_params = grid_search_kNN.best_params_

# make prediction on the test set
y_pred = grid_search_kNN.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# convert the report to a DataFrame
df_report = pd.DataFrame(report).transpose()

# add the column with the cell types
df_report.reset_index(inplace=True)
df_report.rename(columns={'index': 'Cell Type'}, inplace=True)

# stop the timer
end_time_kNN = time.time()

# calculate the execution time
duration_kNN = end_time_kNN - start_time_kNN

# print the execution time
print(f"Execution time: {int(duration_kNN // 60)} minutes and {int(duration_kNN % 60)} seconds" if duration_kNN >= 60 else f"Execution time: {int(duration_kNN)} seconds\n")

# print best hyperparameters
print("Best parameters: {}".format(best_params) + "\n")

# print classification report
print(df_report.to_string(index=False))

Execution time: 37 seconds

Best parameters: {'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}

      Cell Type  precision   recall  f1-score      support
CD14+ Monocytes   0.974290 0.982181  0.978220  1852.000000
  CD20+ B cells   0.995388 0.993151  0.994269  9345.000000
   CD4+ T cells   0.993519 0.993847  0.993683  9101.000000
   CD8+ T cells   0.997521 0.974806  0.986033  2064.000000
    Other cells   0.939716 0.968331  0.953809  1642.000000
       accuracy   0.989293 0.989293  0.989293     0.989293
      macro avg   0.980087 0.982463  0.981203 24004.000000
   weighted avg   0.989427 0.989293  0.989333 24004.000000


In [32]:
# Naive Bayes

# start the timer
start_time_NB = time.time()

# create a pipeline with data standardization and Naive Bayes
pipeline_NB = Pipeline([
    ('scaler', MinMaxScaler()),
    ('nb', GaussianNB())
])

# define the hyperparameters to be optimized
param_grid_NB = {
    'nb__var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]
}

# create a GridSearchCV object
grid_search_NB = GridSearchCV(pipeline_NB, param_grid_NB, cv=5, scoring='accuracy', n_jobs=-1)

# fit the model
grid_search_NB.fit(X_train, y_train)

# keep best parameters
best_params = grid_search_NB.best_params_

# make prediction on the test set
y_pred = grid_search_NB.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# convert the report to a DataFrame
df_report = pd.DataFrame(report).transpose()

# add the column with the cell types
df_report.reset_index(inplace=True)

# stop the timer
end_time_NB = time.time()

# calculate the execution time
duration_NB = end_time_NB - start_time_NB

# print the execution time
print(f"Execution time: {int(duration_NB // 60)} minutes and {int(duration_NB % 60)} seconds" if duration_NB >= 60 else f"Execution time: {int(duration_NB)} seconds\n")

# print best hyperparameters
print("Best parameters: {}".format(best_params) + "\n")

# print classification report
print(df_report.to_string(index=False))

Execution time: 1 seconds

Best parameters: {'nb__var_smoothing': 1e-06}

          index  precision   recall  f1-score      support
CD14+ Monocytes   0.860710 0.837473  0.848933  1852.000000
  CD20+ B cells   0.968018 0.981380  0.974653  9345.000000
   CD4+ T cells   0.996448 0.986375  0.991386  9101.000000
   CD8+ T cells   0.975214 0.991279  0.983181  2064.000000
    Other cells   0.911166 0.899513  0.905302  1642.000000
       accuracy   0.967422 0.967422  0.967422     0.967422
      macro avg   0.942311 0.939204  0.940691 24004.000000
   weighted avg   0.967248 0.967422  0.967287 24004.000000


In [33]:
# Random Forest

# start the timer
start_time_RF = time.time()

# create a pipeline with data standardization and Random Forest
pipeline_RF = Pipeline([
    ('scaler', MinMaxScaler()),
    ('rf', RandomForestClassifier())
])

# define the hyperparameters to be optimized
param_grid_RF = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_features': ['sqrt', 'log2'],
    'rf__max_depth': [None, 5, 10, 20]
}

# create a GridSearchCV object
grid_search_RF = GridSearchCV(pipeline_RF, param_grid_RF, cv=5, scoring='accuracy', n_jobs=-1)

# fit the model
grid_search_RF.fit(X_train, y_train)

# keep best parameters
best_params = grid_search_RF.best_params_

# make prediction on the test set
y_pred = grid_search_RF.predict(X_test)

# generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# convert the report to a DataFrame
df_report = pd.DataFrame(report).transpose()

# add the column with the cell types
df_report.reset_index(inplace=True)
df_report.rename(columns={'index': 'Cell Type'}, inplace=True)

# stop the timer
end_time_RF = time.time()

# calculate the execution time
duration_RF = end_time_RF - start_time_RF

# print the execution time
print(f"Execution time: {int(duration_RF // 60)} minutes and {int(duration_RF % 60)} seconds" if duration_RF >= 60 else f"Execution time: {int(duration_RF)} seconds\n")

# print best hyperparameters
print("Best parameters: {}".format(best_params) + "\n")

# print classification report
print(df_report.to_string(index=False))

Execution time: 12 minutes and 32 seconds
Best parameters: {'rf__max_depth': None, 'rf__max_features': 'log2', 'rf__n_estimators': 300}

      Cell Type  precision   recall  f1-score      support
CD14+ Monocytes   0.987514 0.982181  0.984840  1852.000000
  CD20+ B cells   0.997542 0.998823  0.998182  9345.000000
   CD4+ T cells   0.998461 0.998132  0.998297  9101.000000
   CD8+ T cells   0.996120 0.995155  0.995637  2064.000000
    Other cells   0.982979 0.984775  0.983876  1642.000000
       accuracy   0.996001 0.996001  0.996001     0.996001
      macro avg   0.992523 0.991813  0.992166 24004.000000
   weighted avg   0.995998 0.996001  0.995999 24004.000000


In [34]:
# save the best model (check whether the file alleady exists)
if not os.path.exists('best_model.joblib'):
    dump(grid_search_LR, 'best_model.joblib')